In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.preprocessing import FunctionTransformer, KBinsDiscretizer, StandardScaler, MinMaxScaler, PolynomialFeatures, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.pipeline import make_pipeline
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [6]:
data_path = "Dataset_Brief.csv"
df = pd.read_csv(data_path)

In [7]:
duplicates = df[df.duplicated(keep=False)]
print(duplicates)
index_to_remove = 581
df = df.drop(index_to_remove)
df = df.reset_index(drop=True)
print(df.shape)

     age   sex    bmi  children smoker     region    charges
195   19  male  30.59         0     no  northwest  1639.5631
581   19  male  30.59         0     no  northwest  1639.5631
(1337, 7)


In [8]:
# df['smoker'] = df['smoker'].replace({'yes': 1, 'no': 0})
# df['sex'] = df['sex'].replace({'female': 1, 'male': 0})
categories = {
    'Underweight': (0, 18.5),
    'Normal Weight': (18.5, 24.9),
    'Overweight': (25, 29.9),
    'Obesity Class I': (30, 34.9),
    'Obesity Class II': (35, 39.9),
    'Obesity Class III': (40, float('inf'))
}

# Fonction pour attribuer la catégorie en fonction de la valeur d'IMC
def categorize_imc(bmi):
    for category, (lower, upper) in categories.items():
        if lower <= bmi <= upper:
            return category

# Appliquer la fonction pour créer la nouvelle colonne 'imc_category'
df['imc_category'] = df['bmi'].apply(categorize_imc)
df = df.drop('bmi', axis=1)
df.to_csv('Cleaned_Dataset.csv', index=False)
df

,age,sex,children,smoker,region,charges,imc_category
0,19,female,0,yes,southwest,16884.92400,Overweight
1,18,male,1,no,southeast,1725.55230,Obesity Class I
2,28,male,3,no,southeast,4449.46200,Obesity Class I
3,33,male,0,no,northwest,21984.47061,Normal Weight
4,32,male,0,no,northwest,3866.85520,Overweight
...,...,...,...,...,...,...,...
1332,50,male,3,no,northwest,10600.54830,Obesity Class I
1333,18,female,0,no,northeast,2205.98080,Obesity Class I
1334,18,female,0,no,southeast,1629.83350,Obesity Class II
1335,21,female,0,no,southwest,2007.94500,Overweight
